In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.13


In [19]:
import pickle
import pandas as pd
import os

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [13]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    # this line is needed for question 2
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

    return df

In [14]:
year = 2023
month = 3

df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [15]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [16]:
print(f'Standard deviation of predictions: {y_pred.std():.2f}')

Standard deviation of predictions: 6.25


In [21]:
# homework - question 02

# Next, write the ride id and the predictions to a dataframe with results.
df_results = pd.DataFrame({
    'ride_id': df['ride_id'],
    'prediction': y_pred
})


# Save the results to a parquet file
# Define output directory and file
output_dir = "data"
output_file = os.path.join(output_dir, f"predictions__{year:04d}-{month:02d}.parquet")


df_results.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

# Get and print file size
file_size = os.path.getsize(output_file)
print(f"Predictions saved to: {output_file}")
print(f"File size: {file_size/1024/1024:.2f} MB") 

Predictions saved to: data/predictions__2023-03.parquet
File size: 65.46 MB
